In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold, cross_val_score
import pandas as pd

In [2]:
data = load_iris()
X = data.data    # Fitur
y = data.target  # Label

# 1. GridSearchCV (Standar)

Cara Kerja:
- Tentukan daftar kombinasi hyperparameter (grid).
- Untuk setiap kombinasi:
    - Dilatih & dievaluasi dengan cross-validation (CV).
    - Kombinasi dengan rata-rata skor CV terbaik dipilih.

In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, None],
}

grid = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, scoring='accuracy')
grid.fit(X, y)

print(grid.best_params_)

{'max_depth': 10, 'n_estimators': 200}


# 2. RandomizedSearchCV

Alternatif dari grid search ketika ruang parameter terlalu besar.

⚙️ Cara Kerja:
- Alih-alih mencoba semua kombinasi, ia mengambil sampel acak sejumlah n_iter dari distribusi nilai yang kamu tentukan.
- Tetap memakai cross-validation.

In [6]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_dist = {
    'n_estimators': randint(100, 1000),
    'max_depth': randint(3, 30),
}

search = RandomizedSearchCV(RandomForestClassifier(), param_dist,
                            n_iter=20, cv=5, scoring='accuracy', random_state=42)
search.fit(X, y)
print(search.best_params_)

{'max_depth': 10, 'n_estimators': 800}


# 3. HalvingGridSearchCV (Successive Halving)
Pendekatan efisien & adaptif, diperkenalkan di scikit-learn 0.24+.

⚙️ Cara Kerja:
- Mulai dengan banyak kombinasi hyperparameter, tapi hanya dengan sedikit data atau iterasi training.
- Kombinasi terbaik “naik ke babak berikutnya” dengan lebih banyak resource.
- Kombinasi buruk dieliminasi cepat.

In [8]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': [50, 100, 200, 400],
    'max_depth': [5, 10, 15, None],
}

halving = HalvingGridSearchCV(RandomForestClassifier(), param_grid, cv=5, scoring='accuracy')
halving.fit(X, y)
print(halving.best_params_)

{'max_depth': 15, 'n_estimators': 200}


# 4. HalvingRandomSearchCV

Gabungan ide randomized search + successive halving.

In [10]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV

param_dist = {
    'n_estimators': randint(100, 1000),
    'max_depth': randint(3, 30),
}

halving_rand = HalvingRandomSearchCV(RandomForestClassifier(),
                                     param_dist, cv=5, factor=2, random_state=42)
halving_rand.fit(X, y)
print(halving_rand.best_params_)

{'max_depth': 23, 'n_estimators': 714}


# 5. Manual GridSearch (Custom Loop)

Kadang peneliti membuat grid search manual untuk kontrol penuh:

In [12]:
from sklearn.model_selection import cross_val_score
import itertools

for n in [100, 200]:
    for depth in [5, 10]:
        model = RandomForestClassifier(n_estimators=n, max_depth=depth)
        scores = cross_val_score(model, X, y, cv=5)
        print(n, depth, scores.mean())

100 5 0.9666666666666668
100 10 0.96
200 5 0.96
200 10 0.9666666666666668


## Perbandingan Jenis GridSearchCV dan Variasinya

| Jenis | Strategi | Evaluasi | Kelebihan | Kekurangan |
|-------|-----------|-----------|------------|-------------|
| **GridSearchCV** | Mencoba *semua kombinasi* hyperparameter | Cross Validation | Akurat, deterministik, mudah direproduksi | Lambat jika ruang parameter besar |
| **RandomizedSearchCV** | Sampling *acak* dari ruang parameter | Cross Validation | Lebih cepat, fleksibel, cocok untuk ruang besar | Hasil bisa bervariasi antar run |
| **HalvingGridSearchCV** | *Successive halving* (bertahap, efisien) | CV bertingkat | Cepat, efisien untuk dataset besar | Masih eksperimen, setup agak kompleks |
| **HalvingRandomSearchCV** | Kombinasi *random search + halving* | CV bertingkat | Lebih cepat & adaptif | Hasil bisa kurang stabil |
| **Manual Loop (Custom)** | Looping & evaluasi manual | CV atau evaluasi manual | Paling fleksibel, bisa dikustomisasi penuh | Tidak otomatis, rawan kesalahan |


“Jenis GridSearchCV” dalam konteks luas berarti beragam metode pencarian hyperparameter berbasis cross-validation.

- Kalau ingin hasil paling akurat → GridSearchCV
- Kalau ingin efisien dan scalable → RandomizedSearchCV atau HalvingGridSearchCV